In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from pprint import pprint
from datetime import datetime, timedelta
from tqdm import tqdm
import polars as pl

from eott_dataset import *
from eott_dataset.raw import *

def confirm():
    return input("continue?").lower() in {"yes", "y"}

# %env set EOTT_DATASET_PATH
ds_dir = get_dataset_root()
print("path:", ds_dir)

In [ ]:
pdf = participant_dataframe().collect()

df = pdf.select("pid", "start_time", "rec_time")
starts: dict[int, datetime] = {pid: t for pid, t, _ in df.iter_rows()}
recs: dict[int, timedelta] = {pid: t for pid, _, t in df.iter_rows()}
del df

pdf.write_parquet(ds_dir / "participant.parquet", compression="uncompressed")
pdf

In [ ]:
if confirm():
    with tqdm(desc="webcam", total=len([*glob_webcam_files()])) as t:
        df = webcam_dataframe(callback=lambda: t.update(1))
        df.sink_parquet(ds_dir / "webcam.parquet", compression="uncompressed")

In [ ]:
if confirm():
    with tqdm(desc="screen", total=len([*glob_screen_files()])) as t:
        df = screen_dataframe(callback=lambda: t.update(1))
        df.sink_parquet(ds_dir / "screen.parquet", compression="uncompressed")

In [ ]:
common_columns = ["pid", "record", "timestamp", "study", "duration", "trusted"]
source_columns: dict[Source, list[str]] = {
    Source.MOUSE: ["event", "page", "mouse", "window", "inner", "outer"],
    Source.SCROLL: ["event", "scroll"],
    Source.INPUT: ["event", "caret", "text"],
    Source.TEXT: ["text"],
    Source.LOG: ["event"],
}

with pl.StringCache():
    for source, df in log_dataset().collect().group_by(["source"]):
        source = source[0]
        assert source is not None

        df = df.select(*common_columns, *source_columns[source])
        path = ds_dir.joinpath(source).with_suffix(".parquet")

        df.write_parquet(path, compression="uncompressed")

In [ ]:
tobii_dataset().collect().write_parquet(ds_dir / "tobii.parquet", compression="lz4")

In [ ]:
dot_dataset().collect().write_parquet(ds_dir / "dot.parquet", compression="uncompressed")

In [ ]:
calibration_dataset().collect().write_parquet(ds_dir / "calibration.parquet", compression="uncompressed")

In [ ]:
trackbox_dataset().collect().write_parquet(ds_dir / "trackbox.parquet", compression="uncompressed")

In [ ]:
EyeTyperDataset().timeline().collect().write_parquet(ds_dir / "timeline.parquet", compression="lz4")

In [ ]:
for source, schema in EyeTyperDataset().describe().items():
    print_schema(source, schema)

In [ ]:
# test files are readable
for path in ds_dir.glob("*.parquet"):
    print(path)
    df = pl.read_parquet(path, use_statistics=False)
    pprint(df.schema)
    print()
    del df